# **Automation** Review Alerts

In [6]:
# Importing default Libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import os
import resend

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

# Hi-resolution Plots and Matplotlib inline
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# Set the maximum number of rows and columns to be displayed
import warnings
warnings.filterwarnings("ignore")

# "magic commands" to enable autoreload of your imported packages
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
data = pd.read_csv('../ai_medreview/data/data.csv')
data.head(2)

,time,rating,free_text,do_better,surgery,pcn,sentiment_free_text,sentiment_score_free_text,sentiment_do_better,sentiment_score_do_better,free_text_len,do_better_len,rating_score,feedback_labels,improvement_labels,submission_id,respondent-id,emotion_free_text,emotion_do_better,free_text_PER,do_better_PER,campaing_id,logic,campaign_rating,campaign_freetext,free_text_qa,do_better_qa
0,2024-01-01 10:51:00,Very good,NaN,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,neutral,0.0,neutral,0.0,4.0,0,5.0,NaN,NaN,mannual,upload,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-01-01 15:20:00,Very good,NaN,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,neutral,0.0,neutral,0.0,5.0,0,5.0,NaN,NaN,mannual,upload,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
import pandas as pd
import os
import json
import resend # Added import
from loguru import logger # Added loguru import

In [24]:
CSV_FILE_PATH = '../ai_medreview/data/data.csv' # <-- **UPDATE THIS PATH**
SENTIMENT_COLUMNS = ['sentiment_free_text', 'sentiment_do_better']
SCORE_COLUMNS = ['sentiment_score_free_text', 'sentiment_score_do_better']
NEGATIVE_SENTIMENT_THRESHOLD = 0.6
TIME_COLUMN = 'time' # Column containing the timestamp/date
TIMEFRAME_HOURS = 1500 # Timeframe in hours to check for recent reviews

# Email configuration
# These should ideally be stored as GitHub Secrets for security.
# RESEND_API_KEY = os.environ.get('RESEND_API_KEY') # <-- Get from GitHub Secrets
FROM_EMAIL = 'AI-MedReview - FFT Alert <hello@attribut.me>' # <-- Fixed sender email as per user instructions
# TO_EMAILS_SECRET_PREFIX = 'email-' # Prefix for surgery email secrets (e.g., email-surgery-name)

# --- Function to identify negative reviews ---
def find_negative_reviews(df):
    """
    Identifies negative reviews based on sentiment columns and score threshold,
    filtered for the last X hours based on TIMEFRAME_HOURS.
    """
    if not all(col in df.columns for col in SENTIMENT_COLUMNS + SCORE_COLUMNS + [TIME_COLUMN]):
        missing_cols = [col for col in SENTIMENT_COLUMNS + SCORE_COLUMNS + [TIME_COLUMN] if col not in df.columns]
        logger.error(f"Missing required columns: {', '.join(missing_cols)}")
        return pd.DataFrame()

    # Convert 'time' column to datetime objects
    try:
        # Attempt to convert to datetime, coercing errors to NaT
        df[TIME_COLUMN] = pd.to_datetime(df[TIME_COLUMN], errors='coerce')

        # Drop rows where datetime conversion failed
        initial_rows = len(df)
        df.dropna(subset=[TIME_COLUMN], inplace=True)
        if len(df) < initial_rows:
            logger.warning(f"Dropped {initial_rows - len(df)} rows due to invalid timestamps in '{TIME_COLUMN}'.")

        # Check if the DataFrame is empty after dropping NaNs
        if df.empty:
            logger.warning("DataFrame is empty after dropping rows with invalid timestamps.")
            return pd.DataFrame()

    except Exception as e:
        logger.error(f"Error processing '{TIME_COLUMN}' column for datetime conversion: {e}")
        return pd.DataFrame()

    # Calculate the time threshold based on the current time and timeframe
    time_threshold = pd.Timestamp.now() - pd.Timedelta(hours=TIMEFRAME_HOURS)

    # Filter for reviews within the specified timeframe
    recent_reviews_df = df[df[TIME_COLUMN] >= time_threshold].copy() # Use .copy() to avoid SettingWithCopyWarning

    # Filter for negative sentiment with score >= threshold in either column
    negative_filtered_df = recent_reviews_df[
        (
            (recent_reviews_df[SENTIMENT_COLUMNS[0]].astype(str).str.lower() == 'negative') &
            (recent_reviews_df[SCORE_COLUMNS[0]] >= NEGATIVE_SENTIMENT_THRESHOLD)
        ) |
        (
            (recent_reviews_df[SENTIMENT_COLUMNS[1]].astype(str).str.lower() == 'negative') &
            (recent_reviews_df[SCORE_COLUMNS[1]] >= NEGATIVE_SENTIMENT_THRESHOLD)
        )
    ].copy() # Use .copy() to avoid SettingWithCopyWarning
    # Removed print(negative_filtered_df) - summary is printed below
    return negative_filtered_df

    # Filter for negative sentiment with score >= threshold in either column
    negative_filtered_df = recent_reviews_df[
        (
            (recent_reviews_df[SENTIMENT_COLUMNS[0]].str.lower() == 'negative') &
            (recent_reviews_df[SCORE_COLUMNS[0]] >= NEGATIVE_SENTIMENT_THRESHOLD)
        ) |
        (
            (recent_reviews_df[SENTIMENT_COLUMNS[1]].str.lower() == 'negative') &
            (recent_reviews_df[SCORE_COLUMNS[1]] >= NEGATIVE_SENTIMENT_THRESHOLD)
        )
    ].copy() # Use .copy() to avoid SettingWithCopyWarning


    return negative_filtered_df

# --- Function to format email content for a surgery ---
def format_email_content(surgery_name, negative_reviews_df):
    """
    Formats the email subject, plain text body, and HTML body for a specific surgery
    based on the negative reviews found.
    """
    num_reviews = len(negative_reviews_df)
    subject = f"AI MedReview: {num_reviews} New Negative Review(s) for {surgery_name}"

    # Plain text body
    text_body = f"The following new negative review(s) were found for {surgery_name} in the last 24 hours:\n\n"
    for index, row in negative_reviews_df.iterrows():
        text_body += f"Review Time: {row.get(TIME_COLUMN, 'N/A')}\n"
        text_body += f"General Feedback Sentiment: {row.get(SENTIMENT_COLUMNS[0], 'N/A')} (Score: {row.get(SCORE_COLUMNS[0], 'N/A')})\n"
        text_body += f"Improvement Suggestions Sentiment: {row.get(SENTIMENT_COLUMNS[1], 'N/A')} (Score: {row.get(SCORE_COLUMNS[1], 'N/A')})\n"
        text_body += f"Feedback: {row.get('free_text', 'N/A')}\n"
        text_body += f"Improvement Suggestion: {row.get('do_better', 'N/A')}\n"
        text_body += "-"*30 + "\n"
    text_body += "Regards,\nAI-MedReview Agent"


    # HTML body
    html_body = f"""
<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <style>
    body {{
      margin: 0;
      padding: 0;
      background-color: #f5f5f4;
      font-family: Arial, sans-serif;
    }}
    .email-container {{
      max-width: 600px;
      margin: auto;
      background-color: #ffffff;
      border: 1px solid #ccc;
      border-radius: 12px;
      padding: 24px;
    }}
    .review-item {{
        margin-bottom: 20px;
        padding-bottom: 20px;
        border-bottom: 1px solid #eee;
    }}
    .review-item:last-child {{
        border-bottom: none;
        padding-bottom: 0;
    }}
    strong {{
        color: #333;
    }}
    u {{
        color: #555;
    }}
    hr {{
        color: #e8e8e8;
    }}
  </style>
</head>
<body><BR><BR>
  <div class="email-container">
    <h1 style="color: #333;">{subject}</h1>
    <p style="color: #555;">
      The following new negative review(s) were found for <strong>{surgery_name}</strong> in the last 24 hours:
    </p><BR>
"""

    for index, row in negative_reviews_df.iterrows():
        html_body += f"""
    <div class="review-item">
        <p><u>Review Time: {row.get(TIME_COLUMN, 'N/A')}</u></p>
        <p><strong><u>Feedback Sentiment:</strong> {row.get(SENTIMENT_COLUMNS[0], 'N/A')} (Score: {row.get(SCORE_COLUMNS[0], 'N/A')})</u></p>
        <p><strong>Feedback</strong>: {row.get('free_text', 'N/A')}</p>
        <p><strong><u>Improvement Suggestions Sentiment:</strong> {row.get(SENTIMENT_COLUMNS[1], 'N/A')} (Score: {row.get(SCORE_COLUMNS[1], 'N/A')})</u></p>
        <p><strong>Improvement Suggestion</strong>: {row.get('do_better', 'N/A')}</p>
    </div>
"""

    html_body += """
    <p style="color: #555;">Regards,<br>AI-MedReview Agent</p><BR>
    <p style="font-size: 8pt; color: #888;">This ia an automated email sent from an AI-MedReview Agent using GitHub Actions, if you don't want to receive this anymore email Jan du Plessis.</p>
  </div>
<br><br></body>
</html>
"""

    return subject, text_body, html_body

# --- Function to send email using Resend Library ---
def send_alert_email(to_emails, subject, text_body, html_body):
    """
    Sends an email using the Resend Python library with both text and HTML bodies.
    """
    logger.info(f"Attempting to send email to: {to_emails}")
    logger.info(f"Subject: {subject}")
    # logger.debug(f"Text Body:\n{text_body}") # Use debug for potentially large bodies
    # logger.debug(f"HTML Body (partial):\n{html_body[:500]}...") # Use debug for potentially large bodies

    params: resend.Emails.SendParams = {
        "from": FROM_EMAIL, # Use the fixed FROM_EMAIL
        "to": [to_emails], # 'to' parameter expects a list of strings
        "subject": subject,
        "text": text_body,
        "html": html_body,
    }

    try:
        email = resend.Emails.send(params)
        logger.info("Email sent successfully.")
        # logger.debug(f"Resend API response: {email}") # Use debug for API response
    except Exception as e:
        logger.error(f"Error sending email: {e}")
        # Depending on requirements, you might want to raise the exception
        # or handle it differently (e.g., logging, retries).
        # For now, just logging the error.


In [25]:
df = pd.read_csv(CSV_FILE_PATH)
df.head()

,time,rating,free_text,do_better,surgery,pcn,sentiment_free_text,sentiment_score_free_text,sentiment_do_better,sentiment_score_do_better,free_text_len,do_better_len,rating_score,feedback_labels,improvement_labels,submission_id,respondent-id,emotion_free_text,emotion_do_better,free_text_PER,do_better_PER,campaing_id,logic,campaign_rating,campaign_freetext,free_text_qa,do_better_qa
0,2024-01-01 10:51:00,Very good,NaN,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,neutral,0.000000,neutral,0.0,4.0,0,5.0,NaN,NaN,mannual,upload,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-01-01 15:20:00,Very good,NaN,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,neutral,0.000000,neutral,0.0,5.0,0,5.0,NaN,NaN,mannual,upload,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-01-01 17:47:00,Very good,NaN,NaN,Earls-Court-Surgery,Brompton-Health-PCN,neutral,0.000000,neutral,0.0,2.0,2,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-01-02 07:58:00,Good,PERSON was amazing kind and helpful The recept...,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,positive,0.937987,neutral,0.0,23.0,0,4.0,Reception Staff Interaction,NaN,mannual,upload,admiration,NaN,['Denise'],NaN,NaN,NaN,NaN,NaN,"{'score': 0.034968744963407516, 'start': 36, '...",NaN
4,2024-01-02 10:44:00,Very good,NaN,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,neutral,0.000000,neutral,0.0,1.0,0,5.0,NaN,NaN,mannual,upload,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
negative_reviews = find_negative_reviews(df)
negative_reviews

,time,rating,free_text,do_better,surgery,pcn,sentiment_free_text,sentiment_score_free_text,sentiment_do_better,sentiment_score_do_better,free_text_len,do_better_len,rating_score,feedback_labels,improvement_labels,submission_id,respondent-id,emotion_free_text,emotion_do_better,free_text_PER,do_better_PER,campaing_id,logic,campaign_rating,campaign_freetext,free_text_qa,do_better_qa
12985,2025-03-03 10:42:09,Good,NaN,Waiting time to get to receptionist is quite long,Health-Partners-at-Violet-Melchett,Brompton-Health-PCN,neutral,0.000000,negative,0.656259,0.0,9,4.0,NaN,Waiting Time,E4Lylq,L6pjxl,NaN,neutral,NaN,NaN,nhs_app24,No,0.0,NaN,NaN,"{'score': 5.3161991672823206e-05, 'start': 0, ..."
12989,2025-03-03 11:02:45,Very poor,Rude person I spoke with on phone,NaN,Health-Partners-at-Violet-Melchett,Brompton-Health-PCN,negative,0.851899,neutral,0.000000,7.0,0,1.0,Communication Effectiveness,NaN,G2LNN2,EYZRrq,neutral,NaN,NaN,NaN,nhs_app24,No,0.0,NaN,"{'score': 2.7763435355154797e-06, 'start': 0, ...",NaN
12997,2025-03-03 12:21:03,Very good,I got an appointment I wanted easily,Please REMOVE ME FROM YOUR MAILING LIST You ar...,Health-Partners-at-Violet-Melchett,Brompton-Health-PCN,positive,0.788460,negative,0.911246,7.0,31,5.0,Accessibility and Convenience,Follow-up and Continuity of Care,d5j6Jy,rWkjyv,desire,neutral,NaN,NaN,nhs_app24,No,0.0,NaN,"{'score': 1.2308164514251985e-05, 'start': 0, ...","{'score': 0.04096866026520729, 'start': 112, '..."
13002,2025-03-03 12:48:49,Don't know,The doctor had to cancel the appt i wonder why...,NaN,Health-Partners-at-Violet-Melchett,Brompton-Health-PCN,negative,0.794228,neutral,0.000000,17.0,0,NaN,Appointment Availability,NaN,PaYYz5,KyazqK,surprise,NaN,NaN,NaN,nhs_app24,Yes,0.0,NaN,"{'score': 0.00030409128521569073, 'start': 34,...",NaN
13013,2025-03-03 14:20:34,Very good,NaN,I don t believe anything could of been done be...,Health-Partners-at-Violet-Melchett,Brompton-Health-PCN,neutral,0.000000,negative,0.864089,0.0,9,5.0,NaN,Accessibility and Convenience,gRZ1VP,ePYEPo,NaN,disapproval,NaN,NaN,nhs_app24,No,0.0,NaN,NaN,"{'score': 0.31259506940841675, 'start': 0, 'en..."
13019,2025-03-03 15:37:18,Don't know,I didn t have an appointment at the GP s I am ...,You should have written down the date at which...,Health-Partners-at-Violet-Melchett,Brompton-Health-PCN,neutral,0.635424,negative,0.671318,11.0,91,NaN,Appointment Availability,Appointment Availability,d5jBod,aoaAX2,surprise,remorse,NaN,NaN,nhs_app24,Yes,1.0,People classify documents under the wrong desc...,"{'score': 9.48059096117504e-05, 'start': 46, '...","{'score': 0.06298483908176422, 'start': 16, 'e..."
13034,2025-03-03 21:58:18,Poor,I was not able to see a dr from or at the prac...,Feeling that someone was taking my concerns se...,Health-Partners-at-Violet-Melchett,Brompton-Health-PCN,negative,0.803814,neutral,0.539552,110.0,8,2.0,Follow-up and Continuity of Care,Follow-up and Continuity of Care,YLML86,EYMQgB,realization,caring,NaN,NaN,nhs_app24,Yes,3.0,NaN,"{'score': 0.0277869813144207, 'start': 410, 'e...","{'score': 0.0030480248387902975, 'start': 0, '..."
13039,2025-03-04 08:27:17,Very poor,I needed to see doctor asap but was told that ...,To be able to at least talk to a doctor,Health-Partners-at-Violet-Melchett,Brompton-Health-PCN,negative,0.800651,neutral,0.682353,52.0,11,1.0,Appointment Availability,Accessibility and Convenience,Wvep5N,qezaNd,neutral,neutral,NaN,NaN,nhs_app24,No,0.0,NaN,"{'score': 0.1003636047244072, 'start': 154, 'e...","{'score': 0.033809345215559006, 'start': 0, 'e..."
13051,2025-03-04 09:43:51,Very good,SMWS is very well run with excellent GP s who ...,The receptionists have a flood of calls to dea...,Stanhope-Mews-Surgery,Brompton-Health-PCN,positive,0.657615,negative,0.828402,57.0,40,5.0,Staff Empathy and Compassion,Reception Staff Interaction,JNVg8o,55VzWv,admiration,neutral,NaN,NaN,nhs_app24,No,0.0,NaN,"{'score': 0.016199130564928055, 'start': 230, ...","{'score': 0.04259660094976425, 'start': 93, 'e..."
130

In [29]:
for name, group in negative_reviews.groupby('surgery'):
    print(name)
    display(group.head())  # Works in Jupyter to show nicely

Demo-Surgery-1


,time,rating,free_text,do_better,surgery,pcn,sentiment_free_text,sentiment_score_free_text,sentiment_do_better,sentiment_score_do_better,free_text_len,do_better_len,rating_score,feedback_labels,improvement_labels,submission_id,respondent-id,emotion_free_text,emotion_do_better,free_text_PER,do_better_PER,campaing_id,logic,campaign_rating,campaign_freetext,free_text_qa,do_better_qa
13102,2025-03-04 22:45:50,Poor,The repeat prescription system at this GP surg...,The surgery should invest in a more user frien...,Demo-Surgery-1,Demo-PCN,negative,0.926003,neutral,0.516652,76.0,52,2.0,Prescriptions and Medication Management,Accessibility and Convenience,lPYqYv,85E7pr,disappointment,approval,NaN,NaN,nhs_app24,No,0.0,NaN,"{'score': 0.010728451423346996, 'start': 259, ...","{'score': 0.12476150691509247, 'start': 243, '..."
13103,2025-03-04 22:46:28,Very poor,I ve never experienced such incompetence when ...,The surgery needs to implement a double check ...,Demo-Surgery-1,Demo-PCN,negative,0.915879,neutral,0.793479,58.0,47,1.0,Prescriptions and Medication Management,Follow-up and Continuity of Care,PabDkV,85E7pr,annoyance,neutral,NaN,NaN,nhs_app24,No,0.0,NaN,"{'score': 0.10537382960319519, 'start': 266, '...","{'score': 0.026915473863482475, 'start': 200, ..."
13104,2025-03-04 22:47:21,Very poor,This surgery s repeat prescription service is ...,The surgery should introduce a tracking system...,Demo-Surgery-1,Demo-PCN,negative,0.939629,neutral,0.760051,63.0,52,1.0,Prescriptions and Medication Management,Follow-up and Continuity of Care,M96DgX,85E7pr,disappointment,neutral,NaN,NaN,nhs_app24,No,0.0,NaN,"{'score': 0.025312529876828194, 'start': 206, ...","{'score': 0.03966720774769783, 'start': 221, '..."
13105,2025-03-04 22:47:53,Neither good nor poor,The repeat prescription process here is a disa...,The surgery should provide clear timelines for...,Demo-Surgery-1,Demo-PCN,negative,0.921557,neutral,0.761132,61.0,46,3.0,Prescriptions and Medication Management,Follow-up and Continuity of Care,lPYq15,85E7pr,disappointment,neutral,NaN,NaN,nhs_app24,Yes,5.0,NaN,"{'score': 0.023065948858857155, 'start': 208, ...","{'score': 0.11619246751070023, 'start': 261, '..."
13106,2025-03-04 22:48:29,Good,Dealing with repeat prescriptions at this surg...,The surgery should modernize their prescriptio...,Demo-Surgery-1,Demo-PCN,negative,0.932236,neutral,0.575779,54.0,48,4.0,Prescriptions and Medication Management,Prescriptions and Medication Management,o45DYX,85E7pr,disappointment,neutral,NaN,NaN,nhs_app24,Yes,4.0,NaN,"{'score': 0.035654932260513306, 'start': 273, ...","{'score': 0.2195766419172287, 'start': 208, 'e..."


Earls-Court-Medical-Centre


,time,rating,free_text,do_better,surgery,pcn,sentiment_free_text,sentiment_score_free_text,sentiment_do_better,sentiment_score_do_better,free_text_len,do_better_len,rating_score,feedback_labels,improvement_labels,submission_id,respondent-id,emotion_free_text,emotion_do_better,free_text_PER,do_better_PER,campaing_id,logic,campaign_rating,campaign_freetext,free_text_qa,do_better_qa
13304,2025-03-15 00:13:26,Good,I find completing the request difficult,Stop the automatic system and speak to a human...,Earls-Court-Medical-Centre,Brompton-Health-PCN,negative,0.761080,negative,0.707086,6.0,10,4.0,Follow-up and Continuity of Care,Patient Respect,GNqrYQ,oQO1z5,disappointment,neutral,NaN,NaN,nhs_app24,Yes,3.0,No,"{'score': 1.5020640603324864e-05, 'start': 30,...","{'score': 0.0061785876750946045, 'start': 0, '..."
13700,2025-03-28 18:24:25,Good,I received feedback within 24 hours I was then...,I do not know what I can do in case of somethi...,Earls-Court-Medical-Centre,Brompton-Health-PCN,neutral,0.759145,negative,0.776506,16.0,41,4.0,Follow-up and Continuity of Care,Appointment Availability,RGGz0qK,MQypLM,neutral,confusion,NaN,NaN,nhs_app24,No,0.0,NaN,"{'score': 0.005861830431967974, 'start': 11, '...","{'score': 0.10483469814062119, 'start': 0, 'en..."
13706,2025-03-28 19:23:21,Neither good nor poor,Last week I came to the surgery in a very bad ...,I hope your staff esp in the reception to be m...,Earls-Court-Medical-Centre,Brompton-Health-PCN,negative,0.815860,neutral,0.553534,84.0,22,3.0,Reception Staff Interaction,Reception Staff Interaction,7RRda89,DEP29p,neutral,optimism,NaN,NaN,nhs_app24,Yes,3.0,Just download it a few days ago so I can't rea...,"{'score': 0.040244895964860916, 'start': 212, ...","{'score': 0.0051093436777591705, 'start': 0, '..."
13831,2025-04-01 14:55:30,Very poor,the doctor said she would speak to the special...,NaN,Earls-Court-Medical-Centre,Brompton-Health-PCN,negative,0.743690,neutral,0.000000,48.0,5,1.0,Follow-up and Continuity of Care,NaN,ZjjK220,AOV1Pk,neutral,NaN,NaN,NaN,nhs_app24,No,0.0,NaN,"{'score': 0.018751021474599838, 'start': 209, ...",NaN
14041,2025-04-11 18:08:56,Very good,The appointment booking process is quite clunk...,A slick fully integrated app would be good The...,Earls-Court-Medical-Centre,Brompton-Health-PCN,positive,0.765248,negative,0.704753,73.0,21,5.0,Staff Empathy and Compassion,Accessibility and Convenience,Me109E8,6G6Vjk,joy,neutral,NaN,NaN,nhs_app24,Yes,3.0,"The first thing one sees on the app under ""Ser...","{'score': 0.03536878526210785, 'start': 161, '...","{'score': 0.07722766697406769, 'start': 73, 'e..."


Earls-Court-Surgery


...

,time,rating,free_text,do_better,surgery,pcn,sentiment_free_text,sentiment_score_free_text,sentiment_do_better,sentiment_score_do_better,free_text_len,do_better_len,rating_score,feedback_labels,improvement_labels,submission_id,respondent-id,emotion_free_text,emotion_do_better,free_text_PER,do_better_PER,campaing_id,logic,campaign_rating,campaign_freetext,free_text_qa,do_better_qa
13051,2025-03-04 09:43:51,Very good,SMWS is very well run with excellent GP s who ...,The receptionists have a flood of calls to dea...,Stanhope-Mews-Surgery,Brompton-Health-PCN,positive,0.657615,negative,0.828402,57.0,40,5.0,Staff Empathy and Compassion,Reception Staff Interaction,JNVg8o,55VzWv,admiration,neutral,NaN,NaN,nhs_app24,No,0.0,NaN,"{'score': 0.016199130564928055, 'start': 230, ...","{'score': 0.04259660094976425, 'start': 93, 'e..."
13061,2025-03-04 10:36:24,Very poor,I had a face to face consultation over 10 mont...,In this case the telephone consultation is not...,Stanhope-Mews-Surgery,Brompton-Health-PCN,negative,0.824654,neutral,0.666604,72.0,9,1.0,Follow-up and Continuity of Care,Accessibility and Convenience,G2oOje,25ovZD,neutral,neutral,NaN,NaN,nhs_app24,Yes,4.0,NaN,"{'score': 0.006608302239328623, 'start': 189, ...","{'score': 0.11253945529460907, 'start': 13, 'e..."
13071,2025-03-04 12:34:06,Neither good nor poor,NaN,I should have insisted more to look into my in...,Stanhope-Mews-Surgery,Brompton-Health-PCN,neutral,0.000000,negative,0.856186,3.0,14,3.0,NaN,Follow-up and Continuity of Care,BP8747,j2zgAY,NaN,disgust,NaN,NaN,nhs_app24,No,0.0,NaN,NaN,"{'score': 0.009194829501211643, 'start': 14, '..."
13084,2025-03-04 14:42:45,Very good,Timely referral to specialist and easy re orde...,Booking vaccinations by phone was frustrating ...,Stanhope-Mews-Surgery,Brompton-Health-PCN,positive,0.786318,negative,0.840689,12.0,13,5.0,Follow-up and Continuity of Care,Vaccinations,a8ZX9b,dVz5Rz,neutral,annoyance,NaN,NaN,nhs_app24,Yes,4.0,NaN,"{'score': 0.00015831847849767655, 'start': 0, ...","{'score': 0.001854925649240613, 'start': 0, 'e..."
13087,2025-03-04 15:16:12,Neither good nor poor,I had to work hard to get appointments but onc...,I have found it very hard to get through to a ...,Stanhope-Mews-Surgery,Brompton-Health-PCN,positive,0.801045,negative,0.780268,27.0,54,3.0,Appointment Availability,Appointment Availability,Dl0GAj,qeN5ak,approval,neutral,NaN,NaN,nhs_app24,Yes,1.0,There is nothing more important than direct co...,"{'score': 0.013641184195876122, 'start': 80, '...","{'score': 0.058472152799367905, 'start': 199, ..."


The-Abingdon-Medical-Practice


,time,rating,free_text,do_better,surgery,pcn,sentiment_free_text,sentiment_score_free_text,sentiment_do_better,sentiment_score_do_better,free_text_len,do_better_len,rating_score,feedback_labels,improvement_labels,submission_id,respondent-id,emotion_free_text,emotion_do_better,free_text_PER,do_better_PER,campaing_id,logic,campaign_rating,campaign_freetext,free_text_qa,do_better_qa
13667,2025-03-27 17:30:40,Good,I was given an emergency appointment and an ho...,Yes I could have been given an appointment bef...,The-Abingdon-Medical-Practice,Brompton-Health-PCN,negative,0.835741,neutral,0.888809,45.0,20,4.0,Appointment Availability,Appointment Availability,WOOrROJ,ZLD5Ee,neutral,neutral,NaN,NaN,nhs_app24,No,0.0,NaN,"{'score': 0.0006278642104007304, 'start': 149,...","{'score': 0.08800824731588364, 'start': 4, 'en..."
13676,2025-03-28 05:00:37,Poor,NaN,Keep passing the puck Midwife it health visito...,The-Abingdon-Medical-Practice,Brompton-Health-PCN,neutral,0.000000,negative,0.681322,3.0,18,2.0,NaN,Follow-up and Continuity of Care,gDD9z8M,Gj7MAQ,NaN,neutral,NaN,NaN,nhs_app24,No,0.0,NaN,NaN,"{'score': 0.0007598748779855669, 'start': 54, ..."
13685,2025-03-28 13:50:05,Very poor,Dr didn t have any experience with babies coul...,Personnel in charge of baby consultations shou...,The-Abingdon-Medical-Practice,Brompton-Health-PCN,negative,0.863784,neutral,0.658525,35.0,10,1.0,Prescriptions and Medication Management,Appointment Availability,obbMXlO,YKgBVd,annoyance,neutral,NaN,NaN,nhs_app24,No,0.0,NaN,"{'score': 0.052837714552879333, 'start': 42, '...","{'score': 0.00697449641302228, 'start': 0, 'en..."
14004,2025-04-11 13:45:56,Neither good nor poor,I understand that the Easter period is busy bu...,An available appointment within 15 days,The-Abingdon-Medical-Practice,Brompton-Health-PCN,negative,0.873945,neutral,0.801943,18.0,6,3.0,Appointment Availability,Appointment Availability,kdb2MGJ,WXbWRe,disappointment,neutral,NaN,NaN,nhs_app24,No,0.0,NaN,"{'score': 0.0659528598189354, 'start': 48, 'en...","{'score': 0.00010871530685108155, 'start': 0, ..."
14018,2025-04-11 14:04:36,Neither good nor poor,Wanted an appointment with any doctor and coul...,NaN,The-Abingdon-Medical-Practice,Brompton-Health-PCN,negative,0.657783,neutral,0.000000,14.0,2,3.0,Appointment Availability,NaN,D4qL4Bb,RB56JJ,desire,NaN,NaN,NaN,nhs_app24,Yes,1.0,Have more doctors in clinic,"{'score': 0.00266307033598423, 'start': 0, 'en...",NaN


The-Chelsea-Practice


,time,rating,free_text,do_better,surgery,pcn,sentiment_free_text,sentiment_score_free_text,sentiment_do_better,sentiment_score_do_better,free_text_len,do_better_len,rating_score,feedback_labels,improvement_labels,submission_id,respondent-id,emotion_free_text,emotion_do_better,free_text_PER,do_better_PER,campaing_id,logic,campaign_rating,campaign_freetext,free_text_qa,do_better_qa
13153,2025-03-07 12:24:23,Very good,Expkain i could not get my mum to the aurgwry ...,NaN,The-Chelsea-Practice,Brompton-Health-PCN,negative,0.830126,neutral,0.000000,27.0,0,5.0,Follow-up and Continuity of Care,NaN,kVQB9o,0jgzAP,neutral,NaN,NaN,NaN,nhs_app24,No,0.0,NaN,"{'score': 0.00011588613415369764, 'start': 77,...",NaN
13166,2025-03-07 14:30:16,Very good,I have been really looked after by the practic...,I was given the wrong beta blockers which coul...,The-Chelsea-Practice,Brompton-Health-PCN,positive,0.840455,negative,0.760125,15.0,16,5.0,Follow-up and Continuity of Care,Prescriptions and Medication Management,78O222,zvAGe8,caring,realization,NaN,NaN,nhs_app24,Yes,5.0,Would be good to re order prescriptions on app,"{'score': 0.013414505869150162, 'start': 83, '...","{'score': 0.07625269144773483, 'start': 22, 'e..."
13174,2025-03-08 15:23:10,Good,NaN,The hospital feels old and dated,The-Chelsea-Practice,Brompton-Health-PCN,neutral,0.000000,negative,0.824999,0.0,6,4.0,NaN,Environment and Ambiance,jO8xO6,KBpDWV,NaN,neutral,NaN,NaN,nhs_app24,Yes,5.0,NaN,NaN,"{'score': 0.0014618035638704896, 'start': 0, '..."
13576,2025-03-21 18:38:39,Very poor,It was the third time I had visited with suici...,Listen to women and learn about perimenopause,The-Chelsea-Practice,Brompton-Health-PCN,negative,0.783766,neutral,0.816832,120.0,7,1.0,Prescriptions and Medication Management,Patient Education and Information,81oxzAP,O9ZxXp,disappointment,neutral,NaN,NaN,nhs_app24,Yes,0.0,NaN,"{'score': 0.01023771334439516, 'start': 363, '...","{'score': 0.008141972124576569, 'start': 0, 'e..."
13872,2025-04-02 08:46:16,Very poor,Luck of communication Behaviour that makes fee...,They need to find away how to treat the patien...,The-Chelsea-Practice,Brompton-Health-PCN,negative,0.707203,negative,0.927843,47.0,17,1.0,Staff Professionalism,Patient Respect,VLLaM4g,jXaxjQ,annoyance,annoyance,NaN,NaN,nhs_app24,Yes,5.0,No,"{'score': 0.07077521830797195, 'start': 214, '...","{'score': 5.268594031804241e-05, 'start': 23, ..."


In [31]:
list(negative_reviews['surgery'].unique())

['Health-Partners-at-Violet-Melchett',
 'Stanhope-Mews-Surgery',
 'Demo-Surgery-1',
 'Kensington-Park-Medical-Centre',
 'The-Chelsea-Practice',
 'Emperors-Gate-Health-Centre',
 'Scarsdale-Medical-Centre',
 'Earls-Court-Medical-Centre',
 'The-Abingdon-Medical-Practice',
 'Earls-Court-Surgery']

In [32]:
negative_reviews.groupby('surgery').count()

,time,rating,free_text,do_better,pcn,sentiment_free_text,sentiment_score_free_text,sentiment_do_better,sentiment_score_do_better,free_text_len,do_better_len,rating_score,feedback_labels,improvement_labels,submission_id,respondent-id,emotion_free_text,emotion_do_better,free_text_PER,do_better_PER,campaing_id,logic,campaign_rating,campaign_freetext,free_text_qa,do_better_qa
surgery,,,,,,,,,,,,,,,,,,,,,,,,,,
Demo-Surgery-1,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,0,0,5,5,5,0,5,5
Earls-Court-Medical-Centre,9,9,9,7,9,9,9,9,9,9,9,8,9,7,9,9,9,7,0,0,9,9,9,6,9,7
Earls-Court-Surgery,2,2,2,0,2,2,2,2,2,2,2,2,2,0,2,2,2,0,0,0,2,2,2,1,2,0
Emperors-Gate-Health-Centre,15,15,15,14,15,15,15,15,15,15,15,15,15,14,15,15,15,14,4,2,15,15,15,6,15,14
Health-Partners-at-Violet-Melchett,69,69,66,60,69,69,69,69,69,69,69,65,66,60,69,69,66,60,6,1,69,69,69,25,66,60
Kensington-Park-Medical-Centre,4,4,4,2,4,4,4,4,4,4,4,4,4,2,4,4,4,2,0,0,4,4,4,1,4,2
Scarsdale-Medical-Centre,7,7,7,0,7,7,7,7,7,7,7,7,7,0,0,0,7,0,0,0,0,0,0,0,7,0
Stanhope-Mews-Surgery,22,22,21,20,22,22,22,22,22,22,22,22,21,20,22,22,21,20,2,1,22,22,22,5,21,20
The-Abingdon-Medical-Practice,5,5,4,4,5,5,5,5,5,5,5,5,4,4,5,5,4,4,0,0,5,5,5,1,4,4


In [33]:
negative_reviews.groupby('surgery').head()

,time,rating,free_text,do_better,surgery,pcn,sentiment_free_text,sentiment_score_free_text,sentiment_do_better,sentiment_score_do_better,free_text_len,do_better_len,rating_score,feedback_labels,improvement_labels,submission_id,respondent-id,emotion_free_text,emotion_do_better,free_text_PER,do_better_PER,campaing_id,logic,campaign_rating,campaign_freetext,free_text_qa,do_better_qa
12985,2025-03-03 10:42:09,Good,NaN,Waiting time to get to receptionist is quite long,Health-Partners-at-Violet-Melchett,Brompton-Health-PCN,neutral,0.000000,negative,0.656259,0.0,9,4.0,NaN,Waiting Time,E4Lylq,L6pjxl,NaN,neutral,NaN,NaN,nhs_app24,No,0.0,NaN,NaN,"{'score': 5.3161991672823206e-05, 'start': 0, ..."
12989,2025-03-03 11:02:45,Very poor,Rude person I spoke with on phone,NaN,Health-Partners-at-Violet-Melchett,Brompton-Health-PCN,negative,0.851899,neutral,0.000000,7.0,0,1.0,Communication Effectiveness,NaN,G2LNN2,EYZRrq,neutral,NaN,NaN,NaN,nhs_app24,No,0.0,NaN,"{'score': 2.7763435355154797e-06, 'start': 0, ...",NaN
12997,2025-03-03 12:21:03,Very good,I got an appointment I wanted easily,Please REMOVE ME FROM YOUR MAILING LIST You ar...,Health-Partners-at-Violet-Melchett,Brompton-Health-PCN,positive,0.788460,negative,0.911246,7.0,31,5.0,Accessibility and Convenience,Follow-up and Continuity of Care,d5j6Jy,rWkjyv,desire,neutral,NaN,NaN,nhs_app24,No,0.0,NaN,"{'score': 1.2308164514251985e-05, 'start': 0, ...","{'score': 0.04096866026520729, 'start': 112, '..."
13002,2025-03-03 12:48:49,Don't know,The doctor had to cancel the appt i wonder why...,NaN,Health-Partners-at-Violet-Melchett,Brompton-Health-PCN,negative,0.794228,neutral,0.000000,17.0,0,NaN,Appointment Availability,NaN,PaYYz5,KyazqK,surprise,NaN,NaN,NaN,nhs_app24,Yes,0.0,NaN,"{'score': 0.00030409128521569073, 'start': 34,...",NaN
13013,2025-03-03 14:20:34,Very good,NaN,I don t believe anything could of been done be...,Health-Partners-at-Violet-Melchett,Brompton-Health-PCN,neutral,0.000000,negative,0.864089,0.0,9,5.0,NaN,Accessibility and Convenience,gRZ1VP,ePYEPo,NaN,disapproval,NaN,NaN,nhs_app24,No,0.0,NaN,NaN,"{'score': 0.31259506940841675, 'start': 0, 'en..."
13051,2025-03-04 09:43:51,Very good,SMWS is very well run with excellent GP s who ...,The receptionists have a flood of calls to dea...,Stanhope-Mews-Surgery,Brompton-Health-PCN,positive,0.657615,negative,0.828402,57.0,40,5.0,Staff Empathy and Compassion,Reception Staff Interaction,JNVg8o,55VzWv,admiration,neutral,NaN,NaN,nhs_app24,No,0.0,NaN,"{'score': 0.016199130564928055, 'start': 230, ...","{'score': 0.04259660094976425, 'start': 93, 'e..."
13061,2025-03-04 10:36:24,Very poor,I had a face to face consultation over 10 mont...,In this case the telephone consultation is not...,Stanhope-Mews-Surgery,Brompton-Health-PCN,negative,0.824654,neutral,0.666604,72.0,9,1.0,Follow-up and Continuity of Care,Accessibility and Convenience,G2oOje,25ovZD,neutral,neutral,NaN,NaN,nhs_app24,Yes,4.0,NaN,"{'score': 0.006608302239328623, 'start': 189, ...","{'score': 0.11253945529460907, 'start': 13, 'e..."
13071,2025-03-04 12:34:06,Neither good nor poor,NaN,I should have insisted more to look into my in...,Stanhope-Mews-Surgery,Brompton-Health-PCN,neutral,0.000000,negative,0.856186,3.0,14,3.0,NaN,Follow-up and Continuity of Care,BP8747,j2zgAY,NaN,disgust,NaN,NaN,nhs_app24,No,0.0,NaN,NaN,"{'score': 0.009194829501211643, 'start': 14, '..."
13084,2025-03-04 14:42:45,Very good,Timely referral to specialist and easy re orde...,Booking vaccinations by phone was frustrating ...,Stanhope-Mews-Surgery,Brompton-Health-PCN,positive,0.786318,negative,0.840689,12.0,13,5.0,Follow-up and Continuity of Care,Vaccinations,a8ZX9b,dVz5Rz,neutral,annoyance,NaN,NaN,nhs_app24,Yes,4.0,NaN,"{'score': 0.00015831847849767655, 'start': 0, ...","{'score': 0.001854925649240613, 'start': 0, 'e..."
13087,2025-03-04 15:16:12,Neither good nor poor,I had to work hard to get appointments but onc...,I have found it very hard to get through to a ...,Stanhope-Mews-Surgery,Brompton-Health-PC